# 大差がつく相手の分布

In [1]:
import pandas as pd
import numpy as np
import re
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from itertools import product
from scipy import stats

%matplotlib inline
plt.style.use('ggplot')
#plt.style.use('seaborn-pastel')
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tq

import matplotlib as mpl
font = {"family":"osaka"}
mpl.rc('font', **font)

sys.path.append("..")
from utils.util_functions import *

pd.set_option('display.max_rows', 500)

In [2]:
#event_df = make_df(data_dir=DETAIL_DATA_DIR_MLB)
#event_df = pd.read_csv(os.path.join(ROOT_DIR, "event_df.csv"), index_col=0, encoding="cp932", dtype=object)
#df_2017 = make_df(end=1944417, data_dir=DETAIL_DATA_DIR_MLB)
#df_2017 = pd.read_csv(os.path.join(ROOT_DIR, "df_2017.csv"), index_col=0, encoding="cp932", dtype=object)
#df_2018 = make_df(start=1944417, data_dir=DETAIL_DATA_DIR_MLB)
#df_2018 = pd.read_csv(os.path.join(ROOT_DIR, "df_2018.csv"), index_col=0, encoding="cp932", dtype=object)
files = os.listdir(DETAIL_DATA_DIR_MLB)[1:]
names = pd.Series(files).apply(lambda x: int(x.split(".")[0]))
files_2017 = names[names<=1944417].apply(lambda x: str(x)+".csv")
files_2018 = names[names>1944417].apply(lambda x: str(x)+".csv")
column = "得点"
teams_mlb = list(set(pd.read_csv(os.path.join(SCORE_DATA_DIR_MLB, "score_mlb.csv"), encoding="cp932", index_col=0).表チーム))
teams_mlb.remove("ア・リーグ")
first_teams = ['Rソックス', 'インディアンス', 'アストロズ', 'ブレーブス', 'ブリュワーズ', 'ドジャース']
second_teams = ['ヤンキース', 'ツインズ', 'アスレチックス', 'ナショナルズ', 'カブス', 'ロッキーズ']
third_teams = ['レイズ', 'タイガース', 'マリナーズ', 'フィリーズ', 'カージナルス', 'Dバックス']
fourth_teams = ['ブルージェイズ', 'Wソックス',  'エンゼルス', 'メッツ', 'パイレーツ', 'ジャイアンツ']
fifth_teams = ['オリオールズ', 'ロイヤルズ', 'レンジャーズ', 'マーリンズ', 'レッズ', 'パドレス']

In [3]:
score_df = pd.read_csv(os.path.join(SCORE_DATA_DIR_MLB, "score_mlb.csv"), encoding="cp932", index_col=0)

In [4]:
def func(team, n=5):
    top_leads = np.array(score_df[(score_df.表チーム==team)]["表得点"] - score_df[(score_df.表チーム==team)]["裏得点"])
    top_opps = np.array(score_df[(score_df.表チーム==team)]["裏チーム"])
    bot_leads = np.array(score_df[(score_df.裏チーム==team)]["裏得点"] - score_df[(score_df.裏チーム==team)]["表得点"])
    bot_opps = np.array(score_df[(score_df.裏チーム==team)]["表チーム"])
    leads = np.concatenate([top_leads, bot_leads])
    opps = np.concatenate([top_opps, bot_opps])
    df = pd.DataFrame([leads, opps], index=["リード", "相手チーム"]).T
    return df[df.リード>=n]

In [5]:
# 大差をつける相手として、有意に分布が違うチームはなさそう
# プロだからある程度接戦になると説明するべきか
for team in teams_mlb:
    opps = score_df[(score_df.表チーム==team) | (score_df.裏チーム==team)][["表チーム", "裏チーム"]].values.flatten()
    opps = opps[opps!=team]
    lead_opps = np.array(func(team).相手チーム)
    print(team, stats.ks_2samp(opps, lead_opps).pvalue)

Rソックス 0.99608320391
エンゼルス 0.948024660279
タイガース 0.493048337129
ナショナルズ 0.112207412744
メッツ 0.790584548442
レイズ 0.34983094668
カブス 0.737445272124
Wソックス 0.413546728639
パドレス 0.281395653032
ブリュワーズ 0.797652966023
Dバックス 0.134287673458
アスレチックス 0.991539385442
ツインズ 0.414559032894
ヤンキース 0.325647485252
マーリンズ 0.995804417892
パイレーツ 0.830513905956
マリナーズ 0.342838945366
ブルージェイズ 0.469885403521
オリオールズ 0.125021564496
レッズ 0.414123522423
インディアンス 0.862600158327
ジャイアンツ 0.674760503036
レンジャーズ 0.971326954903
ドジャース 0.0671422037667
カージナルス 0.169417258144
アストロズ 0.99899319312
ブレーブス 0.504889391146
ロイヤルズ 0.946974592358
ロッキーズ 0.994925038863
フィリーズ 0.353858647478


# positive vs negative

In [6]:
lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team], score_end=1, how="own")))

In [7]:
improved_list = [elem for elem in lst if is_improved(elem[0][0], elem[1][0])]
deteriorated_list = [elem for elem in lst if is_deteriorated(elem[0][0], elem[1][0])]

In [8]:
common_case_list = list(set([elem[1][0] for elem in improved_list]) & set([elem[1][0] for elem in deteriorated_list]))

In [12]:
case = common_case_list[0]
#[elem[2][1]-elem[1][1] for elem in improved_list if elem[1][0]==case]

In [38]:
imp_proba = (pd.Series([elem[1][0] for elem in improved_list]).value_counts()/len(pd.Series([elem[1][0] for elem in improved_list])))[common_case_list]
det_proba = (pd.Series([elem[1][0] for elem in deteriorated_list]).value_counts()/len(pd.Series([elem[1][0] for elem in deteriorated_list])))[common_case_list]

In [41]:
#det_proba

In [10]:
res_dict = dict()
for case in common_case_list:
    imp_score = np.mean([elem[2][1]-elem[1][1] for elem in improved_list if elem[1][0]==case])
    det_score = np.mean([elem[2][1]-elem[1][1] for elem in deteriorated_list if elem[1][0]==case])
    res_dict[case] = (imp_score, det_score)

In [29]:
pd.DataFrame(list(res_dict.values()), index=res_dict.keys(), columns=["imp", "det"])#.describe()

,imp,det
2101,0.197531,0.277778
2110,0.196581,0.236641
1100,0.087607,0.058824
2000,0.023256,0.026204
2111,0.613208,0.441176
2010,0.227586,0.185185
1110,0.207373,0.230769
1011,0.590909,0.375000
2001,0.250000,0.148148
1010,0.131579,0.121951


In [37]:
# positive = is_improved
sum(imp_proba * pd.DataFrame(list(res_dict.values()), index=res_dict.keys(), columns=["imp", "det"])["imp"])

0.1281842818428184

In [39]:
# negative = is_deteriorated
sum(det_proba * pd.DataFrame(list(res_dict.values()), index=res_dict.keys(), columns=["imp", "det"])["det"])

0.0617911615174032